# Распознавание разнообразных блюд

 1. Подготовка окружения. Устанавливаю необходимые библиотеки.

In [2]:
import os
import random
import numpy as np
import torch
import torchvision.models as models
from torchvision import transforms
from sklearn.neighbors import NearestNeighbors
from PIL import Image
import pickle
from tqdm import tqdm

In [3]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2. Подготовка данных. Собиру полные пути ко всем изображениям.

In [4]:
data_dir = 'flowers'
image_paths = []
for root, _, files in os.walk(data_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Исправлена опечатка (была точка с запятой)
            image_paths.append(os.path.join(root, file))

3. Трансформации: изображения фиксированного размера, нормализуем значения пикселей, стандартизация значений пикселей.

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Исправлено std (было 0)
])

4. Подготавила предобученную модель ResNet50 для извлечения признаков (feature extraction) из изображений. 

In [6]:
model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])  # Удаляем последний слой
model = model.to(device)
model.eval()

/home/dasha/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dasha/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


5. Извлекаю эмбеддинги. Список features содержит вектора признаков (2048-мерные для ResNet50) для всех успешно обработанных изображений.

In [7]:
features = []
valid_paths = []

for path in tqdm(image_paths, desc="Extracting features"):
    try:
        img = Image.open(path).convert('RGB')
        img_t = transform(img).unsqueeze(0).to(device)
        with torch.no_grad():
            feature = model(img_t).squeeze().cpu().numpy()
        features.append(feature)
        valid_paths.append(path)
    except Exception as e:
        print(f"Ошибка обработки {path}: {e}")

Extracting features: 100%|██████████████████| 4317/4317 [05:33<00:00, 12.94it/s]


In [8]:
np.save('models/resnet_features.npy', np.array(features))
with open('models/image_paths.pkl', 'wb') as f:
    pickle.dump(valid_paths, f)

6. Создам и сохраню модель для поиска ближайших соседей (Nearest Neighbors) на основе извлеченных признаков изображений. 

In [9]:
nbrs = NearestNeighbors(n_neighbors=5, metric='cosine').fit(features)
with open('models/nbrs.pkl', 'wb') as f:
    pickle.dump(nbrs, f)

In [10]:
print("Модель успешно обучена и сохранена в папке models/")

Модель успешно обучена и сохранена в папке models/


7. Pезультаты поиска похожих изображений.

In [ ]:
import matplotlib.pyplot as plt

def search_similar(query_path, n=5):
    """Поиск n похожих изображений"""
    img = Image.open(query_path).convert('RGB')
    img_t = transform(img).unsqueeze(0).to(device)
    
    with torch.no_grad():
        feature = model(img_t).squeeze().cpu().numpy()
    
    dists, idxs = nbrs.kneighbors([feature], n_neighbors=n)
    return {image_paths[i]: float(1-d) for d,i in zip(dists[0], idxs[0])}

def show_results(query, results):
    plt.figure(figsize=(15,3))
    plt.subplot(1,6,1)
    plt.imshow(Image.open(query)); plt.axis('off')
    for i,(path,sim) in enumerate(results.items(),2):
        plt.subplot(1,6,i)
        plt.imshow(Image.open(path))
        plt.title(f"{sim:.2f}"); plt.axis('off')
    plt.show()

query = random.choice(image_paths)
results = search_similar(query)
show_results(query, results)